In [32]:
from bs4 import BeautifulSoup
import json

In [2]:
soup = BeautifulSoup(open("resources/Ergebnisse - EvaP.html"), "html.parser")

In [46]:
course_evaluation = {}

term_mapping = {
    "WS": 0,
    "SS": 1
}

# find all "a" and "div" elements of class "results-grid-row heading-row"
for heading_row in soup.find_all(lambda tag: tag.name in ['a', 'div'] and 'results-grid-row' in tag.get('class', [])):
    try:
        # find the name of the course which is a spand of class "evaluation-name"
        course_name = heading_row.find("span", class_="evaluation-name").text.strip()
        # find the year of evaluation results in a div of class "text-center semester-short-name"
        course_semester = heading_row.find("div", class_="text-center semester-short-name").text.strip()
        # find the evaluation result in a div of class "badge-grade"
        course_grade = heading_row.find("div", class_="badge-grade").text.strip() or "0,0"
        course_grade = float(course_grade.replace(",", "."))

        # if the course name is already in the dictionary, check if the year is newer
        if course_name.lower() in course_evaluation:
            # match the year with a regex
            candidate_year = int(course_semester[-2:])
            candidate_term = term_mapping.get(course_semester[:2])
            
            record_semester = course_evaluation.get(course_name.lower()).get("semester")
            record_year = int(record_semester[-2:])
            record_term = term_mapping.get(record_semester[:2])

            # if the year is newer, replace the old record
            if candidate_year > record_year:
                course_evaluation[course_name.lower()] = {"grade": course_grade, "semester": course_semester}
            elif candidate_year == record_year and candidate_term > record_term:
                course_evaluation[course_name.lower()] = {"grade": course_grade, "semester": course_semester}
        else:
            # store the result in a dictionary
            course_evaluation[course_name.lower()] = {"grade": course_grade, "semester": course_semester}
    except AttributeError:
        continue

display(course_evaluation.items())

dict_items([('einführungstage bachelor', {'grade': 1.5, 'semester': 'WS 23/24'}), ('einführungstage master', {'grade': 1.8, 'semester': 'WS 23/24'}), ('vorkurs', {'grade': 1.4, 'semester': 'WS 23/24'}), ('3d web development', {'grade': 0.0, 'semester': 'SS 23'}), ('3d-computergrafik ii', {'grade': 1.6, 'semester': 'SS 23'}), (': (algos ∪ data structures) ∩ theory', {'grade': 1.2, 'semester': 'SS 23'}), ('academic writing for science', {'grade': 1.3, 'semester': 'SS 23'}), ('advanced competitive programming', {'grade': 1.5, 'semester': 'SS 23'}), ('advanced cryptography', {'grade': 1.1, 'semester': 'SS 23'}), ('advanced medical machine learning seminar', {'grade': 0.0, 'semester': 'SS 23'}), ('advanced track design thinking (d-school)', {'grade': 1.5, 'semester': 'SS 23'}), ('ai-supported medical image annotation platform', {'grade': 1.6, 'semester': 'SS 23'}), ('algorithm auditing via statistical hypothesis testing', {'grade': 2.0, 'semester': 'SS 23'}), ('algorithmic game theory', {'g

In [47]:
# Convert the dictionary to a JSON string
json_data = json.dumps(course_evaluation)

# Write the JSON string to a file
with open('course_evaluation.json', 'w') as json_file:
    json_file.write(json_data)